### num_rows 기준 정하고 그에 해당하는 시퀀스 모두 제거하는 코드

- 코드설명
1. del_idx 컬럼의 값을 가진 csv file 불러오기 (bitcoin_prepro_v2.ipynb에서 작업)
2. num_rows 기준에 해당하는 값과 그 값이 포함된 시퀀스 제외한 최종 시퀀스 생성
3. 시퀀스 정규화(MinMaxScale) 및 tensor 형태로 변환
4. 모든 sequence 기준(20, 40, ... 320)에 따른 pickle 파일 생성 및 저장

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [3]:
file_path = '../../data/' 
df = pd.read_csv(file_path + 'bitcoin_data_num_rows_gt_5.csv')
df = df.sort_values(by='window_start', ascending=True) # 시간순 정렬
print(df.shape)
df.head()

(149559, 85)


,window_start,window_end,num_rows,lowest_return,highest_return,high_low_gap,trade_vol,volume_power,beginning_price,ending_price,...,ob_end_bs_14,ob_end_bias_0,ob_end_bias_1,ob_end_bias_4,ob_end_bidask_spread,ob_end_liq_0,ob_end_liq_1,ob_end_liq_4,highest_possible_return,del_idx
0,2022-12-16 21:05:30,2022-12-16 21:06:00,14,0.000000,0.000089,0.000089,1.468656,0.747351,22568000.0,22570000.0,...,1.467714,5.470422,10.649683,3.235541,2.0,0.001693,0.002198,0.002412,1.000000,0
1,2022-12-16 21:06:00,2022-12-16 21:06:30,10,0.000000,0.000089,0.000089,0.567585,0.027857,22568000.0,22570000.0,...,0.143039,4.224361,14.918538,3.856600,2.0,0.000531,0.001064,0.001471,1.000000,0
2,2022-12-16 21:06:30,2022-12-16 21:07:00,24,-0.000576,0.000044,0.000620,1.677093,0.146635,22570000.0,22570000.0,...,0.271898,17.677511,9.697905,1.106227,14.0,0.000449,0.000536,0.001821,0.999778,0
3,2022-12-16 21:07:00,2022-12-16 21:07:30,22,-0.000044,0.000443,0.000488,2.439677,0.751995,22557000.0,22567000.0,...,0.640898,95.630870,3.371113,1.367349,2.0,0.000416,0.000480,0.001422,0.999911,0
4,2022-12-16 21:07:30,2022-12-16 21:08:00,24,-0.000443,0.000000,0.000443,2.345821,-0.915608,22565000.0,22555000.0,...,0.081040,0.114815,0.828364,0.068175,10.0,0.000311,0.000560,0.003454,0.999911,0


In [3]:
# for verification
# verification_row = df[df['window_start'] == '2022-12-16 21:05:30'][['window_start',
#                                                                     'lowest_return',
#                                                                     'highest_return',
#                                                                     'highest_possible_return',
#                                                                     'returns_next10m']]
# verification_row

#### 전처리) 변수 선택 
- 종속변수 단일화
- 시간 관련 변수 제거
- 종속변수 맨 뒤로 이동(맨 뒤에 값을 제외하고 정규화 진행하기 위함)

In [4]:
# 종속변수 리스트
target_var_lst = ['returns', 'returns_next10m', 'realized_vol_next10m']
target_var = 'returns_next10m' # 종속변수

# 시퀀스 생성 전 필요없는 컬럼 삭제
df.drop(columns=['window_start', 'window_end','num_rows', 'time_id'], inplace=True) # 시간 관련 변수

# target을 제외한 나머지 종속변수 삭제
cols_to_drop = [var for var in target_var_lst if var != target_var]
df.drop(columns=cols_to_drop, inplace=True) # 종속변수

# 종속변수를 데이터 프레임 맨 뒤로 옮기기
cols = df.columns.tolist()
cols = [col for col in cols if col != 'returns_next10m'] + ['returns_next10m'] # 종속변수 맨 뒤로
df = df[cols]
df.columns

Index(['lowest_return', 'highest_return', 'high_low_gap', 'trade_vol',
       'volume_power', 'beginning_price', 'ending_price', 'lowest_price',
       'highest_price', 'ob_end_ap_0', 'ob_end_as_0', 'ob_end_bp_0',
       'ob_end_bs_0', 'ob_end_ap_1', 'ob_end_as_1', 'ob_end_bp_1',
       'ob_end_bs_1', 'ob_end_ap_2', 'ob_end_as_2', 'ob_end_bp_2',
       'ob_end_bs_2', 'ob_end_ap_3', 'ob_end_as_3', 'ob_end_bp_3',
       'ob_end_bs_3', 'ob_end_ap_4', 'ob_end_as_4', 'ob_end_bp_4',
       'ob_end_bs_4', 'ob_end_ap_5', 'ob_end_as_5', 'ob_end_bp_5',
       'ob_end_bs_5', 'ob_end_ap_6', 'ob_end_as_6', 'ob_end_bp_6',
       'ob_end_bs_6', 'ob_end_ap_7', 'ob_end_as_7', 'ob_end_bp_7',
       'ob_end_bs_7', 'ob_end_ap_8', 'ob_end_as_8', 'ob_end_bp_8',
       'ob_end_bs_8', 'ob_end_ap_9', 'ob_end_as_9', 'ob_end_bp_9',
       'ob_end_bs_9', 'ob_end_ap_10', 'ob_end_as_10', 'ob_end_bp_10',
       'ob_end_bs_10', 'ob_end_ap_11', 'ob_end_as_11', 'ob_end_bp_11',
       'ob_end_bs_11', 'ob_end_ap_12', 'ob

In [5]:
# 예전 버전
# def del_sequence(df, seq_len):
#     # 시퀀스 데이터를 저장할 리스트
#     sequences = []
    
#     for start_idx in range(len(df) - seq_len + 1): # 데이터 프레임을 순회하며 시퀀스 생성
#         end_idx = start_idx + seq_len
#         sequence = df.iloc[start_idx:end_idx]

#         # 시퀀스 내에 del_idx가 1인 행이 있다면, 해당 시퀀스를 제외
#         if sequence['del_idx'].sum() == 0:
#             # del_idx 열을 제외하고 시퀀스 추가
#             sequences.append(sequence.drop('del_idx', axis=1).values)

#     return sequences


#### sequence 함수 생성
- sequence 별로 데이터 나누고 정규화 진행

In [6]:
# num_rows가 n개 이하인 것에 해당하는 sequence를 제거하는 함수

# 스케일 추가 버전
def del_sequence(df, seq_len):
    # 시퀀스 데이터를 저장할 리스트
    sequences = []
    scaler = MinMaxScaler()
    
    for start_idx in range(len(df) - seq_len + 1):  # 데이터 프레임을 순회하며 시퀀스 생성
        end_idx = start_idx + seq_len
        sequence = df.iloc[start_idx:end_idx]
        
        # 시퀀스 내에 del_idx가 1인 행이 있다면, 해당 시퀀스를 제외
        if sequence['del_idx'].sum() == 0:
            # 예측하고자 하는 마지막 피처의 값을 제외하고 스케일링
            scaled_sequence = scaler.fit_transform(sequence.drop(columns=['del_idx', sequence.columns[-1]]))

            # 스케일링된 시퀀스에 예측하고자 하는 마지막 피처의 값을 추가
            scaled_sequence_with_target = pd.concat([pd.DataFrame(scaled_sequence), sequence[sequence.columns[-1]].reset_index(drop=True)], axis=1)
            
            # 최종 시퀀스 추가
            sequences.append(scaled_sequence_with_target.values)
    
    return sequences

In [7]:
# sequence 제거 수행
seq_len = 20 # 시퀀스 길이
sequences = del_sequence(df, seq_len=seq_len)
print(f"shape of sequence data: {np.array(sequences).shape}")

shape of sequence data: (76616, 160, 78)


In [8]:
# for verification
#sequences[0][0]

#### train test split

In [7]:
import torch
import sequence as sq
import platform

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# 운영체제별 device 설정
os_name = platform.system()
if os_name == 'Windows':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"이 PC는 윈도우 운영 체제입니다: {device} is available")
elif os_name == 'Darwin':
    device = torch.device("mps" if torch.backends.mps.is_available else "cpu")
    print(f"이 PC는 맥(OS X) 운영 체제입니다: {device} is available")
else:
    print(f"이 PC는 다른 운영 체제입니다: {os_name}")

# 데이터 불러오기
file_path = '../../data/' # 경로 설정
df = pd.read_csv(file_path + 'bitcoin_data_num_rows_gt_5.csv')
#df = df.iloc[:10000]
df['returns_next10m'] = df['returns_next10m'].apply(lambda x: 0 if x <= 0 else 1) # 종속변수 이진분류화
df = df.sort_values(by='window_start', ascending=True) # 시간순 정렬

# sequence length를 기준으로 sequence 데이터 생성
seq_len = 20 # 20, 40, 80, 160, 320
X, y = sq.create_sequence(df, seq_len=seq_len) # 사용자 정의 함수
# Tensor화
X = torch.FloatTensor(X).to(device)
y = torch.FloatTensor(y).to(device)
print('Full Data Size:', X.size(), y.size())

# split (60% / 20% / 20%)
train_split = int((X.size(0)) * 0.8)
#valid_split = int((X.size(0)) * 0.8)

X_train_seq = X[:train_split]
#X_val_seq = X[train_split:valid_split]
X_test_seq = X[train_split:]
y_train_seq = y[:train_split]
#y_val_seq = y[train_split:valid_split]
y_test_seq = y[train_split:]

print('Train Size:', X_train_seq.size(), y_train_seq.size())
#print('Valid Size:', X_val_seq.size(), y_val_seq.size())
print('Test Size:', X_test_seq.size(), y_test_seq.size())

이 PC는 맥(OS X) 운영 체제입니다: mps is available
Full Data Size: torch.Size([111077, 20, 77]) torch.Size([111077, 1])
Train Size: torch.Size([88861, 20, 77]) torch.Size([88861, 1])
Test Size: torch.Size([22216, 20, 77]) torch.Size([22216, 1])


In [9]:


# device = torch.device("mps")
# # X, y값 나누기

# sequences = np.array(sequences)

# # X와 y를 분리
# X = torch.FloatTensor(sequences[:, :, :-1]).to(device) # 마지막 시퀀스와 마지막 컬럼을 제외한 나머지
# y = torch.FloatTensor(sequences[:, -1, -1]).view(-1, 1).to(device) # 각 시퀀스의 마지막 행, 마지막 컬럼의 값

# # split 정의 후 train, test split
# split = int(len(sequences)*0.8) # 70%를 학습데이터로
# x_train_seq = X[:split]
# y_train_seq = y[:split]
# x_test_seq = X[split:]
# y_test_seq = y[split:]

# # 결과 확인
# print(f'X shape: {X.size()}')  # X의 형태 확인
# print(f'y shape: {y.size()}')  # y의 형태 확인
# print(f'x_train shape: {x_train_seq.size()}, y_train shape: {y_train_seq.size()}')
# print(f'x_test shape: {x_test_seq.size()}, y_test shape: {y_test_seq.size()}')

X shape: torch.Size([76616, 160, 77])
y shape: torch.Size([76616, 1])
x_train shape: torch.Size([53631, 160, 77]), y_train shape: torch.Size([53631, 1])
x_test shape: torch.Size([22985, 160, 77]), y_test shape: torch.Size([22985, 1])


#### pickle 파일 생성 및 저장

In [9]:
# Pickle 모듈 사용하여 리스트 파일로 내보내기
import os
import pickle
file_path = '../../data/'

# 폴더가 없으면 생성
if not os.path.exists(file_path):
    os.makedirs(file_path)

# 리스트 파일로 저장
with open(f'{file_path}x_train_seq_{seq_len}.pkl', 'wb') as file: # x_train
    pickle.dump(X_train_seq, file)

with open(f'{file_path}y_train_seq_{seq_len}.pkl', 'wb') as file: # y_train
    pickle.dump(y_train_seq, file)

with open(f'{file_path}x_test_seq_{seq_len}.pkl', 'wb') as file: # x_test
    pickle.dump(X_test_seq, file)

with open(f'{file_path}y_test_seq_{seq_len}.pkl', 'wb') as file: # y_test
    pickle.dump(y_test_seq, file)

print(f'{seq_len} sequence data export finished')

20 sequence data export finished


In [12]:
# 불러오기
with open(f'{file_path}x_train_seq_{seq_len}.pkl', 'rb') as file:
     loaded_lst= pickle.load(file)

In [ ]:
# Mac에서 돌리면 kernel 뻑남..
# tmp_seq = []

# # 가정: create_sequence 함수는 이미 정의되어 있으며, df 데이터프레임과 sequence_length 값을 입력으로 받음
# sequence_lengths = [20, 40, 80, 160, 320]

# # 리스트 컴프리헨션을 사용하여 각 sequence_length에 대한 결과의 길이 계산
# tmp_seq = [len(create_sequence(df, length)) for length in sequence_lengths]

# # 결과를 pandas DataFrame으로 변환
# result_df = pd.DataFrame({
#     'sequence_length': sequence_lengths,
#     'length': tmp_seq
# })

# result_df

In [ ]:
# data export
#result_df.to_csv(file_path+'seq_data_counts.csv', index=False)

In [5]:
import sequence

In [6]:
X, y = sequence.create_sequence(df, 20)
print(X.size(), y.size())

torch.Size([111077, 20, 77]) torch.Size([111077, 1])
